<a href="https://colab.research.google.com/github/bahag-hammers/GoogleLab/blob/main/%5BDatathon%5D%C2%A0Lab_2_Data_Exploration_and_BQML_Model_Training_%5BMAKE_A_COPY%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2:** Data Exploration and Model Training

### **Step 1:** Preparation and Authentication
Set the right parameters and authenticate your user.

In [ ]:
#@title Parameters

project_id = " hub-datathon-poc-a25d" #@param {type:"string"}
team_name  = "E" #@param {type:"string"}

location   = "us" #This is currently necessary
dataset_name = "datathon_ds_{}".format(team_name)

In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import Client, QueryJobConfig
from google.colab import auth
auth.authenticate_user()

client = bigquery.Client(project=project_id)

### **Step 2:** Data Exploration
Get a feeling of the data and explore different facets and features

To remember: This is how the final training dataset looks like

| User Demographic Data | User Behavioral Data | Label
| --- | --- | --- |
| country | cnt_user_engagement | churned
| operating_system | cnt_level_start_quickplay |  |
| language | cnt_level_end_quickplay |  |
|  |  cnt_level_complete_quickplay |  |
|  |  cnt_level_reset_quickplay |  |
|  |  cnt_post_score |  |
|  |  cnt_spend_virtual_currency |  |
|  |  cnt_ad_reward |  |
|  |  cnt_challenge_a_friend |  |
|  |  cnt_completed_5_levels |  |
|  |  cnt_use_extra_steps |  |
|  |  user_first_engagement |  | **bold text**

In [ ]:
query = f"""SELECT * FROM `{dataset_name}.cc_train_dataset`"""
job = client.query(query)
df = job.to_dataframe()
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=2)
plt.subplots_adjust(right=2.0)

# which country do users belong to?
df.country.value_counts(sort=True)[:5].plot(kind='pie',ax=axes[0])
# which operating system is widely used?
df.operating_system.value_counts().plot(kind='pie', ax=axes[1])

In [ ]:
# How many users churned?
df.churned.value_counts().plot(kind='pie')

## **Step 3:** Train Propensity Churn Model

Which model should we use for our use case?
* List of classification [models](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-glm) in BQ to explore (Tip: Keep in mind the training time for more complex models ⏰)
  *  Logistic Regression
  *  Boosted Trees
  *  Deep Neural Networks
  *  Wide and Deep Networks
  *  AutoML tables

Feel free to switch the BQ UI to run your queries there.
Save your progress by saving your queries through the editor.

**Now it is your turn to train a model usign BQML !!**
Here is an example in the documentation:
https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-glm#training_a_logistic_regression_model_with_specified_weights


In [ ]:
%%bigquery --project $project_id

CREATE MODEL `<model-name>`
OPTIONS(MODEL_TYPE='<type>',
        INPUT_LABEL_COLS = ['<label>'])
AS
SELECT * EXCEPT (user_pseudo_id)
FROM `<train-dataset>`

### **Feature Engineering**: How about we create new features from the **user_first_engagement** timestamp field?

Let's use the [TRANSFORM](https://cloud.google.com/bigquery-ml/docs/bigqueryml-transform) method in BigQuery ML to extract month, day of year, day of week and hour from the **user_first_enagagement** field.

Using the TRANSFORM clause, you can specify all preprocessing during model creation. The preprocessing is **automatically** applied during the prediction and evaluation phases of machine learning 😎

Hint: Use the [EXTRACT](https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions#extract) and [TIMESTAMP_MICROS](https://cloud.google.com/bigquery/docs/reference/standard-sql/timestamp_functions#timestamp_micros) functions

In [ ]:
%%bigquery --project $project_id
CREATE MODEL `<model-name>`

TRANSFORM(
  EXTRACT(<..>) as month,
  EXTRACT(<..>) as julianday,
  EXTRACT(<..>) as dayofweek,
  EXTRACT(<..>) as hour,
  * EXCEPT(user_first_engagement, user_pseudo_id)
)

OPTIONS(MODEL_TYPE='<model-type>',
        INPUT_LABEL_COLS = ['<label>'])
AS
SELECT *
FROM `<train-dataset>`

## Step 5: Evaluate Model

**Let**'s evaluate our model performance on the previously created evaluation view. Evaluation datasets are a useful way to tune your models accuracy before testing in production.

In [ ]:
%%bigquery --project $project_id
SELECT
  *
FROM
  ML.EVALUATE(MODEL `<model-name>`,
    (
    SELECT * EXCEPT(user_pseudo_id)
    FROM
      `<eval-dataset>`)
    )